### 03_Creating Multimodal RAG using Amazon Managed Knowledge Bases

Knowledge Bases for Amazon Bedrock leverage Retrieval Augmented Generation (RAG), a technique that harnesses customer data stores to enhance responses generated by foundation models. Knowledge bases allow agents to access existing customer data repositories without extensive administrator overhead. To connect a knowledge base to your data, you specify an S3 bucket as the data source. By employing knowledge bases, applications gain enriched contextual information, streamlining development through a fully-managed RAG solution. This level of abstraction accelerates time-to-market by minimizing the effort of incorporating your data into agent functionality and it optimizes cost by negating the necessity for continuous model retraining to leverage private data.
Knowledge Base Preparation



### Use Case Description

We will imagine that we are a Hotel Booking Assistant who helps user with some relevent information abotu the hotel such as finding where the Casino is, to what has been the performance of the hotel in certain quarter based on financial reports to basic room and hotel fare info.

For this we would need a RAG System that has these relevent information about a particular hotel. For this excersise we will assume we are ABC Hotel and we will laod some documents which have some information about the hotel. This information will include not just text but also other charts, graph information. 


### Architecture Overview

![./images/RAG-diagram.gif](images/RAG-diagram.gif)




### 1. Create the KB Setup

**Step 1:** Navigate to the [Amazon Bedrock > Knowledge base > Create knowledge](https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/knowledge-bases/create-knowledge-base) base console as shown:

![images/kb/kb_starter.png](images/kb/kb_starter.png)

**Step 2**: Next, lets select Create Knowledge Base > Knowledge Base with Vector Store 

**Step 3**:Next, lets provide knowledge base details such as KB name, Description etc.

In below illustration, we are giving 
- **KB Name**: `knowledge-base-hotel-info` and feel free to put some relevent description
- **IAM Role**: Let KB create an IAM role with all needed permissions.
- **DataSource**: Next we choose S3 as our Data source, where we will add some menus to use as RAG data source later in the notebook.
![images/kb/kb_id_iam_role.png](images/kb/kb_id_iam_role.png)

![images/kb/ds_id.png](images/kb/ds_id.png)

For self-hosted workshop, you need to create 2 S3 buckets, you can call them `mmu-workshop-********` and `mmu-workshop-tmp-********`

<div class="alert alert-block alert-info">
<b>Note: S3 URI for Data Source</b>- ⚠️ For s3 location for Data Source choose the bucket where you will store the multimodal pdf files. If you are using in workshop you should see a bucket with similar name mmu-workshop-********  and create a partition in there to seperate our data such as `mm-data`
So our overall S3 URI becomes like

    
  `s3://mmu-workshop-********/mm-data`
<br>
<b>Note: S3 URI for Multimodal Storage</b>- ⚠️ For s3 location for Multimodal Storage(we will show this below) we will create a seperate bucket where the parsed images will be stored. If you are using in workshop you should see a bucket with similar name - mmu-workshop-tmp-********
So our overall S3 URI becomes like

    
  `s3://mmu-workshop-tmp-********`
</div>


To get our KB stood up. For ref please look at the screenshot attached.

![images/kb/kb_fill_in.png](images/kb/kb_fill_in.png)

**Step 4**: **Configure Data Source**
Now, lets configure the data source(S3, in this case) for this to happen we need to provide details as described below
- **S3 Destination**: The S3 URI where our Multimodal Files are located
- For **Parsing Strategy** we will choose _"**FM Parsing with Claude Haiku 3**"_- This means that sonent would be used to parse the multimodal content and summarize the images before passing it to generator model, where we will be using Amazon Nova.
- We will also show how to use **Bedrock Data Automation as a parser** to fully manage the experience of preprocessing multimodal files

![images/kb/kb_configure2.png](images/kb/kb_configure2.png)

**Step 5: Select Embedding Model and Configure Vector Store**

- Next, we will select the embedding models to create the vector representation of the multimodal datasets and we will choose **Titan Embedding Model v2**

- For Vector Store, we will keep the default choice to **Open Search Serverless**

- We would also be required to have an intermidiatery s3 location to drop extracted images from multimodal files so those can be refrenced at teh generation time as shown below.

![images/kb/kb_embedding.png](images/kb/kb_embedding.png)

**Step 6: Review and Create the Knowledge Base**

Finally, lets review the details enter, and click submit to create a Multimodal Knowledge base

Once the Knowledge base is created (this generally takes 4-5 mins) you would see the following message on screen

![images/kb/kb_successmsg.png](images/kb/kb_successmsg.png)



### This overall process takes roughly 10 mins

In [1]:
%store -r

In [4]:
import boto3
import os
import json
import os
import boto3
from botocore.exceptions import ClientError
import pprint
import random
import time
from retrying import retry

boto3_session = boto3.session.Session(region_name='us-west-2')
pp = pprint.PrettyPrinter(indent=2)

### Ignore any pip install dependency errors you may witness

In [3]:
!pip install --upgrade -q -r requirements.txt  --ignore-installed

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.29.0 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.36.15 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-mul

### 2. Key Information

Once the Knowledge Base is created lets note down the following key information in below cell


<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure to add Knowledge Base Name, Id, Data Source ID and the IAM role name created below
</div>

![images/kb/kb_id_iam_role.png](images/kb/kb_id_iam_role.png)

![images/kb/ds_id.png](images/kb/ds_id.png)


In [5]:
## ⚠️ ⚠️ replace below values with the created knowledge base and data source
kb_name = "knowledge-base-quick-start-ruiliang"
kb_id = "0P698YVCTM"
ds_id = "VLWEBP7VAT"
kb_iam_role_name = "AmazonBedrockExecutionRoleForKnowledgeBase_v24y2"

In [20]:
kb_region_name='us-west-2'
nova_reigon_name = 'us-east-1'

bedrock_agent_client = boto3_session.client('bedrock-agent', 
                                            region_name=kb_region_name)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', 
                                            region_name=kb_region_name)

get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id)

get_ds_response = bedrock_agent_client.get_data_source(knowledgeBaseId = kb_id, dataSourceId = ds_id)
account_id = boto3.client("sts").get_caller_identity()["Account"]


### 3. Ingest the Hotel Information Files into the S3 location 

In [8]:
def interactive_sleep(seconds: int):
    dots = ''
    for i in range(seconds):
        dots += '.'
        print(dots, end='\r')
        time.sleep(1)

### We will use 2 S3 buckets 
1. **Data Source Input S3 Bucket**: This s3 bucket will serve as an input for creating our Data Source which will create a Vector Database using OpenSearch Serverless. For this we will use the pre-created bucket of the format
`mmu-workshop-<ACCOUNT_ID>-*****`

2. **Multimodal Storage Bucket**: This s3 bucket will be used to write and read any extracted images from multimodal documents that needs to be refrenced while answering questions related to images. For this we will use a pre-created bucket of the format
`mmu-workshop-tmp-<ACCOUNT_ID>-*****`

**Note**: We will be syncing in "mm-data" partition, if you are syncing somewhere else please make the appropriate modifications

In [9]:
# Upload data to s3 to the bucket that was configured as a data source to the knowledge bas

bucket_name = "sagemaker-us-west-2-710299592439"
s3_path = "genai/knowledge-base-source-data"

s3_client = boto3.client("s3", region_name=kb_region_name)
def uploadDirectory(path, bucket_name, s3_path):
    for root, dirs, files in os.walk(path):
        for file in files:
            local_file_path = os.path.join(root, file)
            s3_key = os.path.join(s3_path, os.path.relpath(local_file_path, path))
            # Upload the file with the new S3 key
            s3_client.upload_file(local_file_path, bucket_name, s3_key)

uploadDirectory('kb_data', bucket_name, s3_path)

### 4. Sync the KB Data Source _[You can also skip this to do from UI by clicking "Sync Data Source"]_

![images/kb/kb_sync.png](images/kb/kb_sync.png)

This may take 5-7 mins to sync

In [10]:
interactive_sleep(30)
ingest_jobs=[]
# Start an ingestion job
try:
    start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb_id, dataSourceId = ds_id)
    job = start_job_response["ingestionJob"]
    job_id = job["ingestionJobId"]
    print(f"Ingestion job started successfully\n")

    while job['status'] not in ["COMPLETE", "FAILED", "STOPPED"]:
        get_job_response = bedrock_agent_client.get_ingestion_job(
          knowledgeBaseId = kb_id,
            dataSourceId = ds_id,
            ingestionJobId = job_id
      )
        job = get_job_response["ingestionJob"]
    pp.pprint(job)
    interactive_sleep(40)
    ingest_jobs.append(job)
except Exception as e:
    print(f"Couldn't start job.\n")
    print(e)

Ingestion job started successfully

{ 'dataSourceId': 'VLWEBP7VAT',
  'ingestionJobId': 'YKMHCKG1TK',
  'knowledgeBaseId': '0P698YVCTM',
  'startedAt': datetime.datetime(2025, 2, 7, 10, 6, 4, 486189, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 3,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 3},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 2, 7, 10, 6, 31, 603795, tzinfo=tzlocal())}


### 5. Update the IAM Policy to include Amazon Nova as generator

In [17]:
def attach_policy_json_to_role(role_name, policy_name, policy_json):
    """
    Attaches a policy JSON directly to an IAM role.

    :param role_name: The name of the IAM role
    :param policy_name: The name to give the new policy
    :param policy_json: The policy document as a JSON string or dictionary
    :return: True if successful, False otherwise
    """
    try:
        # Create an IAM client
        iam_client = boto3.client('iam')

        # Ensure policy_json is a string
        if isinstance(policy_json, dict):
            policy_json = json.dumps(policy_json)

        # Create the policy
        response = iam_client.create_policy(
            PolicyName=policy_name,
            PolicyDocument=policy_json
        )

        # Get the ARN of the newly created policy
        policy_arn = response['Policy']['Arn']

        # Attach the policy to the role
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )

        print(f"Successfully created policy {policy_name} and attached it to role {role_name}")
        return True

    except Exception as e:
        print(f"Error attaching policy JSON to role: {str(e)}")
        return False

# Example usage
policy_name = "NovaProModelPolicy2"
policy_json = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "BedrockInvokeModelStatement",
            "Effect": "Allow",
            "Action": [
                "bedrock:*"
            ],
            "Resource": [
                f"arn:aws:bedrock:{nova_reigon_name}::foundation-model/{PRO_MODEL_ID}",
            ]
        }
    ]
}

attach_policy_json_to_role(kb_iam_role_name, policy_name, policy_json)

Error attaching policy JSON to role: An error occurred (EntityAlreadyExists) when calling the CreatePolicy operation: A policy called NovaProModelPolicy2 already exists. Duplicate names are not allowed.


False

### 6. Test the KB retrieve and retrieve and generate

### Using RetrieveAndGenerate API
Behind the scenes, RetrieveAndGenerate API converts queries into embeddings, searches the knowledge base, and then augments the foundation model prompt with the search results as context information and returns the FM-generated response to the question. For multi-turn conversations, Knowledge Bases manage short-term memory of the conversation to provide more contextual results.

The output of the RetrieveAndGenerate API includes the generated response, source attribution as well as the retrieved text chunks.

In [12]:
import s3fs
import ipywidgets as widgets
from PIL import Image as PILImage
import io


def ask_bedrock_llm_with_knowledge_base(query: str, model_arn: str, kb_id: str) -> str:
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': query
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': model_arn
            }
        },
    )

    return response

fs = s3fs.S3FileSystem()

## Function to print retrieved response
def print_response(response):
#structure 'retrievalResults': list of contents. Each list has ['ResponseMetadata', 'citations', 'output', 'sessionId']
    generated_text = response["output"]["text"]
    ref_ref_location_lst=[]
    ref_ref_location_lst.append({"generated_text":generated_text})
    for num, chunk in enumerate(response['citations']):   
        ref_locations=[]
        for i, ref in enumerate (chunk['retrievedReferences']):
            data_dict = {"ref_location": ref['location'], 
                        "ref_metadata": ref['metadata']['x-amz-bedrock-kb-source-uri']}
            if 'x-amz-bedrock-kb-byte-content-source' in ref['metadata'].keys():
                 data_dict["ref_image"] = ref['metadata']['x-amz-bedrock-kb-byte-content-source']
            ref_locations.append(data_dict)
        ref_ref_location_lst.append({"chunk_details":ref_locations})
    return ref_ref_location_lst



import ipywidgets as widgets
import s3fs
from PIL import Image as PILImage
import io


def create_tree_widget(data, s3=None):
    s3 = s3 or s3fs.S3FileSystem(anon=False)
    main_accordion = widgets.Accordion()
    
    for i, item in enumerate(data):
        subchildren = []
        
        # Always add Generated Text first
        if 'generated_text' in item:
            text_widget = widgets.Textarea(
                value=str(item['generated_text']),
                disabled=True,
                layout=widgets.Layout(width='500px', height='200px')
            )
            subchildren.append(text_widget)
        
        # Then add Chunk Details
        if 'chunk_details' in item:
            chunk_accordion = widgets.Accordion()
            chunk_children = []
            
            for chunk in item['chunk_details']:
                chunk_subchildren = []
                
                for key, value in chunk.items():
                    if key == 'ref_image' and isinstance(value, str) and value.startswith('s3://'):
                        try:
                            with s3.open(value, 'rb') as f:
                                img = PILImage.open(f).resize((400, 400))
                                img_byte_arr = io.BytesIO()
                                img.save(img_byte_arr, format='PNG')
                                img_widget = widgets.Image(
                                    value=img_byte_arr.getvalue(),
                                    format='png',
                                    width=400,
                                    height=400
                                )
                            chunk_subchildren.append(img_widget)
                        except Exception as e:
                            chunk_subchildren.append(widgets.Label(f"Image Error: {e}"))
                    else:
                        chunk_subchildren.append(widgets.Label(f"{key}: {json.dumps(value)}"))
                
                chunk_item_accordion = widgets.Accordion(children=tuple(chunk_subchildren))
                for k, child in enumerate(chunk_subchildren):
                    chunk_item_accordion.set_title(k, list(chunk.keys())[k])
                
                chunk_children.append(chunk_item_accordion)
            
            chunk_accordion = widgets.Accordion(children=tuple(chunk_children))
            for j, child in enumerate(chunk_children):
                chunk_accordion.set_title(j, f'Chunk {j+1}')
            
            subchildren.append(chunk_accordion)
        
        # Create item accordion with correct titles
        item_accordion = widgets.Accordion(children=tuple(subchildren))
        item_accordion.set_title(0, 'Generated Text')
        if len(subchildren) > 1:
            item_accordion.set_title(1, 'Chunk Details')
        
        main_accordion.children += (item_accordion,)
        main_accordion.set_title(i, f'Item {i}')
    
    return main_accordion

### Using Textual Information Retrieval

In [24]:
query = "What is the check-in and check-out time at ABC Grand hotel?"
cross_region_inference = True

if cross_region_inference == True:
    model_arn = f'arn:aws:bedrock:{kb_region_name}:{account_id}:inference-profile/us.{PRO_MODEL_ID}'
else:
    model_arn = f'arn:aws:bedrock:{nova_reigon_name}::foundation-model/{PRO_MODEL_ID}'

response = ask_bedrock_llm_with_knowledge_base(query, model_arn, kb_id)
response_data = print_response(response)
tree_widget = create_tree_widget(response_data)
display(tree_widget)

Accordion(children=(Accordion(children=(Textarea(value='The check-in time at ABC Grand hotel is 3:00 PM, and t…

### Using Multimodal search to find information

In [25]:
query = "Is there a Fitness Room at ABC Grand? If so, where is it located ?"

cross_region_inference = True

if cross_region_inference == True:
    model_arn = f'arn:aws:bedrock:{kb_region_name}:{account_id}:inference-profile/us.{PRO_MODEL_ID}'
else:
    model_arn = f'arn:aws:bedrock:{nova_reigon_name}::foundation-model/{PRO_MODEL_ID}'
    
response = ask_bedrock_llm_with_knowledge_base(query, model_arn, kb_id)
response_data = print_response(response)
tree_widget = create_tree_widget(response_data)
display(tree_widget)

Accordion(children=(Accordion(children=(Textarea(value='The model cannot find sufficient information to answer…

### Chart and Graph Understanding

In [26]:
query = "What is the revenue in 2022 versus 2023?"
cross_region_inference = True

if cross_region_inference == True:
    model_arn = f'arn:aws:bedrock:{kb_region_name}:{account_id}:inference-profile/us.{PRO_MODEL_ID}'
else:
    model_arn = f'arn:aws:bedrock:{nova_reigon_name}::foundation-model/{PRO_MODEL_ID}'

response = ask_bedrock_llm_with_knowledge_base(query, model_arn, kb_id)
response_data = print_response(response)
tree_widget = create_tree_widget(response_data)
display(tree_widget)

Accordion(children=(Accordion(children=(Textarea(value='The revenue in 2023 was 558.1 million USD, which is an…

In [ ]:
%store kb_name
%store kb_id
%store ds_id
%store account_id
%store bucket_name